In [1]:
import numpy as np  # for array stuff and random
import random
import matplotlib.pyplot as plt  # for graphing our mean rewards over time
from matplotlib import style  # to make pretty charts because it matters.
import os
import shutil

import gym
from gym import spaces
from gym import envs
from gym.utils import seeding, EzPickle

from stable_baselines3 import A2C

In [2]:
seed = 543 # Do not change this
def fix(env, seed):
    env.seed(seed)
    env.action_space.seed(seed)
    np.random.seed(seed)
    random.seed(seed)

In [3]:
copyDir = 'C:/Users/johnn/Desktop/file3'# 複製資料夾
putDir = 'C:/Users/johnn/Desktop/loop test'# 複製要放的資料夾

for root, dirs, files in os.walk(copyDir):
    # 針對檔案單層複製
    for fileName in files:

        # 取得要複製檔案的完整路徑
        fileP = root + '/' + fileName
        # 取得要複製檔案放置的完整路徑
        newFP = putDir + '/' + fileName

        # 進行複製 (有檔案會直接覆蓋)
        shutil.copyfile(
            fileP,
            newFP
            )

In [4]:
env = gym.make('pumpingAllocation-v7')
fix(env, seed)
#model = A2C.load("a2c_test3")

S_max = env.max_storage
D_min, D_max = np.min(env.demands), np.max(env.demands)
In_min, In_max = np.min(env.rivers), np.max(env.rivers)

print(S_max, D_min, D_max, In_min, In_max)

650 2473.63 13876.7 17.27 1991.91


In [5]:
env = gym.make('pumpingAllocation-v7')
fix(env, seed)
#model = A2C.load("a2c_test3")

SW, GW, Sup, D, inflow = [], [], [], [], []
deficit=[]
S, h = [], []
num_satisfy, num_unsatisfy, num_sat_follow_unsat = 0, 0, 0

for i in range(3):
    print('----------test',i+1,'start-------------')
    state = env.reset()
    total_step=0
    while True:
        #action, _states = model.predict(state)
        action = [5, 0]
        #action = [random.randint(0,10), random.randint(0,10)]
        next_state, reward, done, info = env.step(action)
        
        #還原標準化
        storage = (state[0]+1)/2*S_max
        #Zmin_before = (state[1]+1)/2*100-100
        #Zmin_after = (next_state[1]+1)/2*100-100
        demand = (state[1]+1)/2*(D_max-D_min+400)+D_min-200
        inf = (state[2]+1)/2*(In_max-In_min)+In_min
        
        print(f'第{total_step+1:2.0f}旬, 水庫水量:{storage:4.0f}, Action:{action}, SW、GW:[{action[0]*16*86.4:3.0f}, {action[1]*120*2.151:3.0f}], 需水量:{demand:4.0f}, Reward: {reward:.1f}')
        print(env.SW, env.GW)
        
        SW.append(action[0]*16*86.4)
        GW.append(action[1]*120*2.151)
        Sup.append(action[0]*16*86.4+action[1]*120*2.151)
        D.append(demand)
        deficit.append(max(0,demand-action[0]*16*86.4-action[1]*120*2.151))
        S.append(storage)
        #h.append(Zmin_before)
        inflow.append(inf)
        '''
        if deficit[-1]>0:
            num_unsatisfy += 1
        else:
            if len(deficit)>1 and deficit[-2]>0:
                num_sat_follow_unsat += 1
                num_satisfy += 1
            else:
                num_satisfy += 1
        '''
        state = next_state
        total_step+=1

        if done:
            break

----------test 1 start-------------
第 1旬, 水庫水量: 400, Action:[5, 0], SW、GW:[6912,   0], 需水量:5031, Reward: -2.0
2249.824 0.0
第 2旬, 水庫水量:   0, Action:[5, 0], SW、GW:[6912,   0], 需水量:5001, Reward: -2.0
2248.992 0.0
第 3旬, 水庫水量:   0, Action:[5, 0], SW、GW:[6912,   0], 需水量:5241, Reward: -2.0
2733.696 0.0
第 4旬, 水庫水量:   0, Action:[5, 0], SW、GW:[6912,   0], 需水量:5679, Reward: -2.0
2725.92 0.0
第 5旬, 水庫水量:   0, Action:[5, 0], SW、GW:[6912,   0], 需水量:5409, Reward: -2.0
2821.8239999999996 0.0
第 6旬, 水庫水量:   0, Action:[5, 0], SW、GW:[6912,   0], 需水量:5469, Reward: -2.0
2580.768 0.0
第 7旬, 水庫水量:   0, Action:[5, 0], SW、GW:[6912,   0], 需水量:6123, Reward: -2.0
2249.856 0.0
第 8旬, 水庫水量:   0, Action:[5, 0], SW、GW:[6912,   0], 需水量:6433, Reward: -2.0
1957.824 0.0
第 9旬, 水庫水量:   0, Action:[5, 0], SW、GW:[6912,   0], 需水量:6113, Reward: -2.0
1953.504 0.0
第10旬, 水庫水量:   0, Action:[5, 0], SW、GW:[6912,   0], 需水量:7421, Reward: -2.0
1836.0 0.0
第11旬, 水庫水量:   0, Action:[5, 0], SW、GW:[6912,   0], 需水量:7381, Reward: -2.0
1804.896 0.0


Exception: MODFLOW did not terminate normally.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
end=180

df = pd.DataFrame({'SW':SW[:end],
                   'GW':GW[:end],
                   'Supply':Sup[:end],
                   'Demand':D[:end],
                   'ith period of tendays':[i for i in range(1,end+1)],
                   'deficit':deficit[:end],
                   'S':S[:end],
                   'h':h[:end],
                   'In':inflow[:end],
                   'deficit_rate':[(b-a)/b*100 for a,b in zip(Sup[:end],D[:end])]
                  }) 


ax = df[['SW','GW']].plot(kind='bar', stacked=True, figsize=(18,10), width=0.7,
                          xlim=[1,180], ylim=[0,15000], xticks=[9*i for i in range(1,21)], fontsize=14)
df['Demand'].plot(kind='line', color='#FF0000', lw=2, xlim=[1,180],legend=True)
df['Supply'].plot(kind='line', style='g.-', lw=2, xlim=[1,180],legend=True)
ax.set_title('SW+GW_Scenerio_{}'.format('random'), fontsize=16)
ax.set_xlabel("tendays", fontsize=14)
ax.set_ylabel("m^3", fontsize=14)
ax.legend()

In [ ]:
print(num_satisfy, num_unsatisfy, num_sat_follow_unsat)
rel = round(num_satisfy/180,2)
res = round(num_sat_follow_unsat/num_unsatisfy,2)
vul = round(np.sum(deficit)/num_unsatisfy/np.sum(D),2)

SI = round((rel*res*(1-vul))**0.3333,2)
print(rel,res,vul)
print(SI)

In [ ]:
ax = df[['S', 'In']].plot(kind='line', lw='2', figsize=(15,9),xlim=[1,180], ylim=[0,26000], xticks=[9*i for i in range(1,21)])
ax.set_title('Storage & Inflow _Scenerio_{}'.format(1), fontsize=16)
ax.set_xlabel("tendays", fontsize=14)
ax.set_ylabel("m^3", fontsize=14)
ax.legend()

In [ ]:
ax = df['GW'].plot(kind='area', color='#00bbff', figsize=(15,9), xlim=[1,180], xticks=[9*i for i in range(1,21)], legend=True)
ax2 = df['h'].plot(kind='line', secondary_y = True,  lw=2, color='#FF0000', legend=True)
ax.set_title('Groundwater Head & Pumping _Scenerio_{}'.format(1), fontsize=16)
ax.set_xlabel("tendays", fontsize=14)
ax.set_ylabel("groundwater pumping (m^3)", fontsize=14)
ax2.set_ylabel("groundwater head (m)", fontsize=14)

In [ ]:
a = 7400
reward = 0
SW=10000
GW=2700

if (a - SW - GW)/a >= 0.3:
    reward -=1
elif (a - SW - GW)/a > 0:
    reward +=0.5
elif (a - SW - GW)/a >= -0.3:
    reward +=1
else:
    reward -=1
    
print(reward)